In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import src.load.load_shape_data as lsd
path_sat_shapefile = "/data_1/ATM/data_1/georef/sat.shp"

# load the shapefile with the images
georef_shape_data = lsd.load_shape_data(path_sat_shapefile)

# get the geometry
georef_shapes = georef_shape_data.geometry

# get the unique ids from the shapes
georef_ids = georef_shape_data['image_id'].tolist()


In [5]:
import src.base.connect_to_database as ctd
import src.load.load_image as li

conn = ctd.establish_connection()

import src.georef.georef_calc as gc

GCalc = gc.GeorefCalc()

while True:
    sql_string = "SELECT * FROM images WHERE image_id LIKE '%V%' ORDER BY RANDOM() LIMIT 1"
    image_data = ctd.execute_sql(sql_string, conn)

    image_id = image_data.iloc[0]['image_id']
    month = image_data.iloc[0]['date_month']

    image = li.load_image(image_id)

    # Filter out geo-referenced images and footprints that don't match the flight path
    filtered_georef_ids = []
    filtered_georef_shapes = []
    for i, geo_id in enumerate(georef_ids):
        if geo_id[2:6] == image_id[2:6]:
            filtered_georef_ids.append(geo_id)
            filtered_georef_shapes.append(georef_shapes[i])

    print("georef", image_id)

    if len(filtered_georef_ids) < 3:
        print("Not enough georef images")
        continue

    transform, residuals, tps, conf = GCalc.georeference(image, image_id, filtered_georef_ids, filtered_georef_shapes)

    break

georef CA181332V0172
